## IMDB Review Exploration & Gated Recurrent Unit (GRU) *Mood* Model

[This is a dataset of 25,000 movies reviews from IMDB](https://ai.stanford.edu/%7Eamaas/data/sentiment/), labeled by sentiment (positive/negative). Reviews have been preprocessed, and each review is encoded as a list of word indexes (integers). For convenience, words are indexed by overall frequency in the dataset.

As a convention, "0" does not stand for a specific word, but instead is used to encode the pad token.

[Documentation](https://www.tensorflow.org/api_docs/python/tf/keras/datasets/imdb/load_data) | [Paper: Learning Word Vectors for Sentiment Analysis](https://ai.stanford.edu/%7Eamaas/papers/wvSent_acl2011.pdf)

In [451]:
import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GRU, Dense
import numpy as np

In [452]:
word_index = imdb.get_word_index() # Just a dictionary, mapping each token to its word

In [453]:
len(word_index)

88584

That's a lotta unique words.

### Top 40 most frequent words

The word indices in the dataset are offset by 4, for special tokens:

`0` For encoding padding token, as mentioned<br>
`1` Start of sequence<br>
`2` Unknown word (OOV or UNK, "out-of-vocabulary" and "unknown")<br>
`3` Index actual words with this index and higher.

In [454]:
pad_char = 0
start_char = 1
oov_char = 2
index_from = 3

inverted_word_index = dict(
    (i + index_from, word) for (word, i) in word_index.items()
)
# Update `inverted_word_index` to include `start_char` and `oov_char`
inverted_word_index[start_char] = "[START]"
inverted_word_index[oov_char] = "[UNK]"

In [455]:
for i in range(20):
    print(f"{inverted_word_index.get(i+4)}, ", end='')

the, and, a, of, to, is, br, in, it, i, this, that, was, as, for, with, movie, but, film, on, 

And that's a lotta stop words. I think we should remove most of them before we train.<br>
But before we do that, let's check out the data some more.

All of the top words make sense from a natural language perspective except one. Probably caused by the html tag `<br>`?<br>
I cannot think of any other reason *br* would be in the reviews. Let's check the next 20 words.

In [456]:
for i in range(20):
    print(f"{inverted_word_index.get(24+i)}, ", end='')

not, you, are, his, have, he, be, one, all, at, by, an, they, who, so, from, like, her, or, just, 

Seems fine. Let's check out the reviews.

In [457]:
(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=10000)

In [458]:
decoded_sentence = ' '.join(inverted_word_index.get(i, '?') for i in train_data[np.random.randint(0, len(train_data))])
decoded_sentence

"[START] warning review contains mild spoilers br br a couple of years back i managed to see the first five films in this franchise and was planning to do an [UNK] of the whole elm st series however just two years on and i find i can't remember enough about them in order to do it \x96 i guess they couldn't have made much of an impression from what i do recall some of the sequels \x96 dream warriors in particular \x96 weren't as bad as is often made out though even the original was no classic generally the predictability of the premise if people fall asleep they get murdered in their dreams doesn't lend itself to narrative tension but while i cannot recall much of the first five films i do know they never [UNK] the depths of freddy's dead br br an indication of how sick of freddy the public was at this point can be judged by the fact that the film was promoted solely on the character's demise the fact that the movie's conclusion is not even hidden but in fact the entire purpose for the 

It's not really readable like this, and I want to continue EDA of the reviews, so I made a function to fetch random samples of the train data and make it a bit more readable.

In [459]:
def get_random_sample(n=25):
    rng = np.random.randint(0, len(train_data))
    review = ' '.join(inverted_word_index.get(i, '?') for i in train_data[rng])
    sentiment = train_labels[rng]
    words = review.split()
    lines = [' '.join(words[i:i+n]) for i in range(0, len(words), n)]
    result = '\n'.join(lines)

    print(f"Sample {rng}:\n\n{result} \n\nLabel: {sentiment}")

get_random_sample()

Sample 2384:

[START] i'm a sucker for a good romance but this one doesn't qualify as either good or a romance i had the plot nailed down
before the credits were through with such poor dialog plot and character development i suggest [UNK] your hour and a half [UNK] i had to
rush out and rent [UNK] for the third time so i could get the bad taste of this one out of my mouth 

Label: 0


In [460]:
get_random_sample()

Sample 14623:

[START] this is an amazing film to watch or show young people aside from a very brief nude scene it gives an interesting glimpse into
colonial rule in africa that you'll rarely find in other films it does bear a superficial similarity to out of africa but without all the
romantic fluff the white french people in [UNK] are fascinating because they don't even seem to regard the natives as people the whites are all
the bosses and they expect black [UNK] without question however unlike real servants you only once hear any of the whites say [UNK] [UNK] and
no other regard is given these people again and again it's like they are pets or slaves as the feelings of the people are never
even considered br br the central [UNK] of this [UNK] is the relationship between the mother [UNK] and her servant [UNK] although at times they
spend a lot of time together and it is only normal that they might begin to have sexual feelings towards each other the white woman
never considers [UNK] o

In [461]:
get_random_sample()

Sample 4638:

[START] i am a current a s l student was forced to watch this movie in class and what i got out of it was
the blatant bias involved in the film the film is obviously [UNK] towards to p o v of the common deaf perception their is no
middle ground also the film didn't make mention or take into account other situations that are also under debate in this topic i e deaf
people who were born hearing and later went deaf is it right or wrong in that instance the film is biased and virtually all in
the opinion of the deaf w a capital d not that this is bad but for it to be a true documentary film is should
attempt to be slightly [UNK] 

Label: 0


Fun read! Let's get cooking.

## Removing stop words

We could use [nltk](https://www.nltk.org), but since we have this slick API we might as well just fetch the data as we want it.

In [462]:
(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=20040, skip_top=16, maxlen=200, start_char=1, oov_char=2)

I cut the most common words at index 16, becuase I want to include the conjunction `but`, since it can add contrast, contrary to the more agreeing `and`.
Not sure if it's a good idea, but let's try!

Well, that was easy "cleaning".

## GRU Model & Training

In [463]:
model = Sequential()
model.add(Embedding(input_dim=20040, output_dim=128, input_length=200))
model.add(GRU(64, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential_23"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_23 (Embedding)    (None, 200, 128)          2565120   
                                                                 
 gru_23 (GRU)                (None, 64)                37248     
                                                                 
 dense_23 (Dense)            (None, 1)                 65        
                                                                 
Total params: 2602433 (9.93 MB)
Trainable params: 2602433 (9.93 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [464]:
train_data = pad_sequences(train_data, maxlen=200)
test_data = pad_sequences(test_data, maxlen=200)

model.fit(train_data, train_labels, epochs=4, batch_size=64, validation_split=0.2)
test_loss, test_acc = model.evaluate(test_data, test_labels)

Epoch 1/4
179/179 [==============================] - 33s 170ms/step - loss: 0.5245 - accuracy: 0.7228 - val_loss: 0.3672 - val_accuracy: 0.8410
Epoch 2/4
179/179 [==============================] - 32s 178ms/step - loss: 0.2384 - accuracy: 0.9079 - val_loss: 0.3438 - val_accuracy: 0.8533
Epoch 3/4
179/179 [==============================] - 31s 171ms/step - loss: 0.1531 - accuracy: 0.9455 - val_loss: 0.4552 - val_accuracy: 0.8624
Epoch 4/4
459/459 [==============================] - 11s 23ms/step - loss: 0.4379 - accuracy: 0.8515


## Inference

In [465]:
new_reviews1 = [
"really enjoyed movie, acting was fantastic but plot was not so great",
"really enjoyed movie, acting was fantastic plot was not so great",
"the, and, a, of, to, is, br, in, it, i, this, that, was, as, for, with, movie, but, film, on, "]

new_reviews2 = [
"It was just decent. I would not watch it again, I could have it running in the background without changing channel",
"It was just decent. I would not watch it again, but I could have it running in the background without changing channel",
"just decent. would not watch again could have running background without changing channel",
"just decent. would not watch again but could have running background without changing channel"]

gpt_positive_reviews = [
"Indulging in a can of tomato soup has never been so satisfying! This culinary delight is a true game-changer, offering a burst of rich, velvety flavor that instantly warms the soul. The vibrant red hue is a testament to the quality of the tomatoes used, providing a visual feast before the first spoonful. The texture is perfectly smooth, creating a luscious and comforting consistency that feels like a hug in a bowl. The harmonious blend of herbs and spices adds layers of complexity to the taste, striking a delightful balance between savory and subtly sweet notes. The aroma that wafts from the steaming soup is a prelude to the culinary masterpiece that awaits. Whether enjoyed as a quick solo lunch or paired with a grilled cheese sandwich for a heartier meal, this tomato soup elevates any dining experience. The convenience of a ready-to-eat can makes it a go-to option for busy days, delivering gourmet quality in the blink of an eye. With a maximum sequence of 200, it's challenging to convey the full depth of the sensory journey that this tomato soup offers. In summary, this can of tomato soup is a must-have pantry essential, delivering a palate-pleasing experience that brings comfort and joy to every spoonful.",
"Riding the tandem bike was an absolute delight from start to finish! The smooth coordination between two riders provided an exhilarating experience that brought joy and laughter throughout the journey. The comfortable seating and ergonomic design made every pedal stroke effortless, allowing us to explore scenic routes without any discomfort. The sturdy frame and responsive handling ensured a safe and secure ride, while the efficient gearing system effortlessly tackled various terrains. With a generous sequence length of 200, this tandem bike review barely scratches the surface of our amazing adventures. Whether cruising along coastal paths or conquering challenging hills, the tandem bike proved to be the perfect companion for creating lasting memories and sharing unforgettable moments. Overall, a fantastic ride that exceeded all expectations!"]

gpt_negative_reviews = [
"Opening the can of tomato soup was a disappointing experience from the very start. The promise of a delightful culinary treat quickly faded as I encountered an overwhelmingly bland aroma that failed to evoke the richness one expects from a tomato soup. Upon tasting, the lackluster flavor profile became painfully apparent – a thin and insipid broth with an absence of the robust tomato essence. The texture further added to the dissatisfaction, as it lacked the desired creaminess and depth that one anticipates in a quality tomato soup. Instead of a comforting consistency, the soup felt watery and devoid of the velvety smoothness that should accompany such a classic dish. The touted blend of herbs and spices turned out to be a mere whisper, contributing little to enhance the overall taste. While convenience is a selling point for canned soups, this particular product left much to be desired. The aftertaste was distinctly metallic, leaving an unpleasant lingering sensation. With a maximum sequence length of 200, it's challenging to convey the full extent of disappointment, but suffice it to say, this can of tomato soup failed to meet even the most basic expectations. A regrettable choice that falls far short of the culinary experience one would hope for.",
"The tandem bike I recently purchased turned out to be a regrettable investment, failing to live up to even the most modest expectations. From the outset, the design proved to be cumbersome and impractical, making the initial setup an arduous task. The promised seamless coordination between riders was anything but, as the tandem bike exhibited an alarming lack of responsiveness to even the most synchronized pedaling efforts. Comfort, a crucial factor in any bike ride, was sorely lacking. The seating arrangement was far from ergonomic, causing discomfort and fatigue within a short period. Instead of a smooth and enjoyable ride, the tandem bike delivered a jarring and unpleasant experience, amplifying every bump and uneven surface on the road. The build quality left much to be desired, with noticeable issues in the frame that raised concerns about safety. Rather than instilling confidence, the tandem bike inspired a constant worry about its structural integrity, overshadowing any attempt to appreciate the outdoor scenery. As for the promised sense of unity and shared enjoyment, it was nowhere to be found. The tandem bike, far from fostering a sense of togetherness, ended up causing frustration and tension between riders. With a maximum sequence length of 200, it's challenging to encapsulate the full extent of disappointment, but suffice it to say, this tandem bike failed to deliver on its promises and left me questioning the decision to purchase it. A regrettable choice for anyone seeking a harmonious and enjoyable biking experience."
]

In [466]:
def predict(reviews):
    for i in range(len(reviews)):
        new_text = reviews[i]

        new_sequence = [imdb.get_word_index().get(word, 0) for word in new_text.split()]
        new_sequence = [index if index < 20040 else 0 for index in new_sequence]
        padded = pad_sequences([new_sequence], maxlen=200)
        prediction = model.predict(padded)

        sentiment = "Positive" if prediction[0, 0] > 0.5 else "Negative"
        print(new_sequence)
        print(reviews[i])
        print(f"Predicted sentiment: {sentiment} (Confidence: {prediction[0, 0] * 100:.2f}%)")
        print()

predict(new_reviews1)

1/1 [==============================] - 0s 326ms/step
[63, 507, 0, 113, 13, 774, 18, 111, 13, 21, 35, 84]
really enjoyed movie, acting was fantastic but plot was not so great
Predicted sentiment: Positive (Confidence: 93.04%)

1/1 [==============================] - 0s 39ms/step
[63, 507, 0, 113, 13, 774, 111, 13, 21, 35, 84]
really enjoyed movie, acting was fantastic plot was not so great
Predicted sentiment: Positive (Confidence: 92.79%)

1/1 [==============================] - 0s 37ms/step
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
the, and, a, of, to, is, br, in, it, i, this, that, was, as, for, with, movie, but, film, on, 
Predicted sentiment: Positive (Confidence: 70.83%)



Third review is like a 'baseline' test? Maybe..? The model has a bias towards positive reviews?

But the first and second are interesting, as in how the word `but` can have an impact. Although we shouldn't jump to conclusions just by this tiny cherry pick.
I guess it's is more of a wildcard. Unlesss we can feature engineer it somehow, I'm not sure the connotation following the word `but` tend to be positive or negative, I mean intuitively it's usually followed by something negative? Maybe it's best just consider it a stop word.

In [467]:
predict(new_reviews2)

1/1 [==============================] - 0s 39ms/step
[0, 13, 40, 0, 0, 59, 21, 103, 9, 0, 0, 97, 25, 9, 617, 8, 1, 975, 206, 2543, 1305]
It was just decent. I would not watch it again, I could have it running in the background without changing channel
Predicted sentiment: Negative (Confidence: 49.38%)

1/1 [==============================] - 0s 39ms/step
[0, 13, 40, 0, 0, 59, 21, 103, 9, 0, 18, 0, 97, 25, 9, 617, 8, 1, 975, 206, 2543, 1305]
It was just decent. I would not watch it again, but I could have it running in the background without changing channel
Predicted sentiment: Positive (Confidence: 50.29%)

1/1 [==============================] - 0s 57ms/step
[40, 0, 59, 21, 103, 171, 97, 25, 617, 975, 206, 2543, 1305]
just decent. would not watch again could have running background without changing channel
Predicted sentiment: Positive (Confidence: 70.97%)

1/1 [==============================] - 0s 40ms/step
[40, 0, 59, 21, 103, 171, 18, 97, 25, 617, 975, 206, 2543, 1305]
just decent. w

But to no surpise, removing the stop words from input improves confidence.

I will test two negative and two positive reviews from something entirely unrelated.

In [468]:
predict(gpt_positive_reviews)

1/1 [==============================] - 0s 43ms/step
[0, 8, 3, 67, 4, 10370, 5769, 44, 112, 74, 35, 0, 0, 0, 3034, 6, 3, 280, 0, 3988, 3, 5563, 4, 0, 0, 6894, 12, 3502, 18395, 1, 0, 0, 5752, 764, 0, 6, 3, 4961, 5, 1, 486, 4, 1, 4805, 0, 3757, 3, 1111, 6529, 156, 1, 83, 0, 0, 10027, 6, 947, 0, 1852, 3, 11030, 2, 12989, 10685, 12, 761, 37, 3, 9141, 8, 3, 0, 0, 0, 3887, 4, 0, 2, 0, 1605, 5885, 4, 4637, 5, 1, 0, 3344, 3, 1914, 2969, 197, 0, 2, 6081, 1044, 0, 0, 0, 12, 0, 36, 1, 8697, 5769, 6, 3, 18696, 5, 1, 0, 988, 12, 0, 0, 507, 14, 3, 1602, 4318, 6800, 39, 8597, 16, 3, 0, 3040, 15457, 15, 3, 0, 0, 11, 10370, 5769, 12142, 98, 13106, 0, 0, 11719, 4, 3, 0, 67, 163, 9, 3, 0, 5447, 15, 2955, 0, 4495, 0, 486, 8, 1, 9374, 4, 32, 0, 0, 3, 7523, 717, 4, 0, 42, 4736, 5, 2830, 1, 365, 1134, 4, 1, 0, 1308, 12, 11, 10370, 5769, 0, 0, 0, 11, 67, 4, 10370, 5769, 6, 3, 0, 0, 0, 4495, 3, 0, 582, 12, 958, 5093, 2, 1802, 5, 172, 0]
Indulging in a can of tomato soup has never been so satisfying! This culina

In [469]:
predict(gpt_negative_reviews)

1/1 [==============================] - 0s 37ms/step
[0, 1, 67, 4, 10370, 5769, 13, 3, 1329, 582, 36, 1, 52, 0, 0, 2336, 4, 3, 1914, 0, 1691, 943, 7859, 14, 0, 6964, 32, 13181, 1901, 0, 12, 1193, 5, 7976, 1, 15356, 28, 6431, 36, 3, 10370, 0, 0, 0, 1, 5129, 6894, 7483, 874, 2146, 1731, 5047, 3, 1520, 2, 5614, 0, 16, 32, 3816, 4, 1, 13113, 10370, 0, 0, 10027, 1034, 1280, 5, 1, 0, 14, 9, 3671, 1, 4627, 0, 2, 1134, 12, 28, 19822, 8, 3, 486, 10370, 0, 0, 4, 3, 12989, 0, 1, 5769, 418, 18290, 2, 4167, 4, 1, 0, 0, 12, 141, 10179, 138, 3, 353, 0, 0, 15550, 3887, 4, 0, 2, 0, 676, 43, 5, 27, 3, 2688, 0, 13517, 114, 5, 6799, 1, 441, 0, 0, 11719, 6, 3, 3485, 210, 15, 10474, 0, 11, 840, 2217, 314, 73, 5, 27, 0, 0, 18078, 13, 8846, 0, 1197, 32, 4006, 8424, 0, 0, 3, 7523, 717, 1612, 4, 0, 42, 4736, 5, 2830, 1, 365, 2823, 4, 0, 18, 4914, 9, 5, 0, 11, 67, 4, 10370, 5769, 1193, 5, 906, 57, 1, 88, 1118, 0, 0, 18431, 1096, 12, 731, 227, 343, 4, 1, 0, 582, 28, 59, 437, 0]
Opening the can of tomato soup was a

It's hilarious how GPT spat out *"With a maximum sequence length of 200, it's challenging to encapsulate the full extent of disappointment."* and still got a positive prediction.

But as we can tell, there are many important words missing. I think using the entire corpus will improve results.

# More data == Better model?

In [470]:
(big_train, big_train_labels), (big_test, big_test_labels) = imdb.load_data(skip_top=20, start_char=1, oov_char=2)

Almost twice as many reviews as in the first dataset.

In [471]:
model2 = Sequential()
model2.add(Embedding(input_dim=88588, output_dim=128, input_length=200))
model2.add(GRU(64, dropout=0.2, recurrent_dropout=0.2))
model2.add(Dense(1, activation='sigmoid'))
model2.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model2.summary()

Model: "sequential_24"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_24 (Embedding)    (None, 200, 128)          11339264  
                                                                 
 gru_24 (GRU)                (None, 64)                37248     
                                                                 
 dense_24 (Dense)            (None, 1)                 65        
                                                                 
Total params: 11376577 (43.40 MB)
Trainable params: 11376577 (43.40 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [472]:
big_train = pad_sequences(big_train, maxlen=200)
big_test = pad_sequences(big_test, maxlen=200)

model2.fit(big_train, big_train_labels, epochs=4, batch_size=64, validation_split=0.2)
test_loss, test_acc = model2.evaluate(big_test, big_test_labels)

Epoch 1/4
313/313 [==============================] - 92s 283ms/step - loss: 0.4552 - accuracy: 0.7729 - val_loss: 0.3260 - val_accuracy: 0.8652
Epoch 2/4
313/313 [==============================] - 88s 281ms/step - loss: 0.2038 - accuracy: 0.9240 - val_loss: 0.3274 - val_accuracy: 0.8722
Epoch 3/4
313/313 [==============================] - 95s 303ms/step - loss: 0.1000 - accuracy: 0.9677 - val_loss: 0.3832 - val_accuracy: 0.8738
Epoch 4/4
782/782 [==============================] - 18s 23ms/step - loss: 0.5431 - accuracy: 0.8493


In [473]:
test_loss, test_acc = model2.evaluate(big_test, big_test_labels)

782/782 [==============================] - 18s 22ms/step - loss: 0.5431 - accuracy: 0.8493


In [474]:
def predict2(reviews):
    for i in range(len(reviews)):
        new_text = reviews[i]

        new_sequence = [imdb.get_word_index().get(word, 0) for word in new_text.split()]
        # new_sequence = [index if index < 20040 else 0 for index in new_sequence]
        padded = pad_sequences([new_sequence], maxlen=200)
        prediction = model2.predict(padded)
        print(prediction)

        sentiment = "Positive" if prediction[0, 0] > 0.5 else "Negative"
        print(new_sequence)
        print(reviews[i])
        print(f"Predicted sentiment: {sentiment} (Confidence: {prediction[0, 0] * 100:.2f}%)")
        print()

In [475]:
predict2(gpt_negative_reviews)

1/1 [==============================] - 0s 359ms/step
[[0.15110855]]
[0, 1, 67, 4, 10370, 5769, 13, 3, 1329, 582, 36, 1, 52, 0, 0, 2336, 4, 3, 1914, 33874, 1691, 943, 7859, 14, 0, 6964, 32, 13181, 1901, 45235, 12, 1193, 5, 7976, 1, 15356, 28, 6431, 36, 3, 10370, 0, 0, 0, 1, 5129, 6894, 7483, 874, 2146, 1731, 5047, 3, 1520, 2, 5614, 38355, 16, 32, 3816, 4, 1, 13113, 10370, 0, 0, 10027, 1034, 1280, 5, 1, 0, 14, 9, 3671, 1, 4627, 0, 2, 1134, 12, 28, 19822, 8, 3, 486, 10370, 0, 0, 4, 3, 12989, 0, 1, 5769, 418, 18290, 2, 4167, 4, 1, 32795, 39199, 12, 141, 10179, 138, 3, 353, 0, 0, 15550, 3887, 4, 31155, 2, 25155, 676, 43, 5, 27, 3, 2688, 0, 13517, 114, 5, 6799, 1, 441, 0, 0, 11719, 6, 3, 3485, 210, 15, 10474, 0, 11, 840, 2217, 314, 73, 5, 27, 0, 0, 18078, 13, 8846, 0, 1197, 32, 4006, 8424, 0, 0, 3, 7523, 717, 1612, 4, 0, 42, 4736, 5, 2830, 1, 365, 2823, 4, 0, 18, 4914, 9, 5, 0, 11, 67, 4, 10370, 5769, 1193, 5, 906, 57, 1, 88, 1118, 0, 0, 18431, 1096, 12, 731, 227, 343, 4, 1, 33874, 582, 28, 

Great improvement on the tomato soup review, but somehow worse predicition on the tandem bike.

In [476]:
stop_words = []
for i in range(20):
    stop_words.append(inverted_word_index.get(i+4))

In [477]:
original_string = gpt_negative_reviews[0]
new = ' '.join(word for word in original_string.split() if word not in stop_words)
print(new)

Opening can tomato soup disappointing experience from very start. The promise delightful culinary treat quickly faded I encountered an overwhelmingly bland aroma failed evoke richness one expects from tomato soup. Upon tasting, lackluster flavor profile became painfully apparent – thin insipid broth an absence robust tomato essence. The texture further added dissatisfaction, lacked desired creaminess depth one anticipates quality tomato soup. Instead comforting consistency, soup felt watery devoid velvety smoothness should accompany such classic dish. The touted blend herbs spices turned out be mere whisper, contributing little enhance overall taste. While convenience selling point canned soups, particular product left much be desired. The aftertaste distinctly metallic, leaving an unpleasant lingering sensation. With maximum sequence length 200, it's challenging convey full extent disappointment, suffice say, can tomato soup failed meet even most basic expectations. A regrettable choi

In [478]:
predict2([new])

1/1 [==============================] - 0s 40ms/step
[[0.27781025]]
[0, 67, 10370, 5769, 1329, 582, 36, 52, 0, 0, 2336, 1914, 33874, 1691, 943, 7859, 0, 6964, 32, 13181, 1901, 45235, 1193, 7976, 15356, 28, 6431, 36, 10370, 0, 0, 0, 5129, 6894, 7483, 874, 2146, 1731, 5047, 1520, 5614, 38355, 32, 3816, 13113, 10370, 0, 0, 10027, 1034, 1280, 0, 3671, 4627, 0, 1134, 28, 19822, 486, 10370, 0, 0, 12989, 0, 5769, 418, 18290, 4167, 32795, 39199, 141, 10179, 138, 353, 0, 0, 15550, 3887, 31155, 25155, 676, 43, 27, 2688, 0, 13517, 114, 6799, 441, 0, 0, 11719, 3485, 210, 10474, 0, 840, 2217, 314, 73, 27, 0, 0, 18078, 8846, 0, 1197, 32, 4006, 8424, 0, 0, 7523, 717, 1612, 0, 42, 4736, 2830, 365, 2823, 0, 4914, 0, 67, 10370, 5769, 1193, 906, 57, 88, 1118, 0, 0, 18431, 1096, 731, 227, 343, 33874, 582, 28, 59, 437, 0]
Opening can tomato soup disappointing experience from very start. The promise delightful culinary treat quickly faded I encountered an overwhelmingly bland aroma failed evoke richness one 

In [479]:
original_string = gpt_negative_reviews[1]
new = ' '.join(word for word in original_string.split() if word not in stop_words)
predict2([new])

1/1 [==============================] - 0s 40ms/step
[[0.80414575]]
[0, 18837, 5474, 0, 1030, 4659, 676, 43, 27, 18431, 0, 3715, 409, 53, 57, 88, 6108, 0, 0, 0, 1589, 2082, 27, 39581, 0, 228, 2402, 6027, 32, 16565, 0, 0, 4404, 13733, 40416, 197, 8231, 230, 0, 18837, 5474, 14078, 32, 12108, 580, 69175, 57, 88, 11958, 55403, 0, 0, 4809, 2364, 98, 5474, 0, 5945, 0, 0, 33859, 12930, 227, 36, 0, 4049, 15480, 19317, 743, 343, 0, 0, 3554, 734, 0, 18837, 5474, 2129, 6318, 4006, 0, 58605, 172, 10930, 4068, 2555, 0, 0, 1700, 486, 314, 73, 27, 0, 6453, 1338, 2119, 2838, 3274, 41, 0, 0, 71, 44813, 0, 18837, 5474, 1578, 1810, 3247, 41, 91, 20201, 0, 72077, 98, 586, 1141, 10214, 0, 0, 4404, 278, 13599, 5342, 0, 1279, 27, 0, 0, 18837, 0, 227, 36, 34553, 278, 0, 1051, 53, 4049, 4182, 1071, 197, 0, 0, 7523, 717, 1612, 0, 42, 4736, 37488, 365, 2823, 0, 4914, 0, 18837, 5474, 1193, 1642, 91, 4928, 314, 69, 7530, 2151, 4435, 0, 0, 18431, 1096, 256, 2984, 24282, 734, 39591, 0]
The tandem bike I recently purc

Removing the stop words so far seem to boost the sentiment in the initial direction.

Maybe if we trained the model on food reviews instead of movies, the canned soup dilemma would've been solved. Let's add a couple more epochs to the second model.

In [480]:
model2.fit(big_train, big_train_labels, epochs=4, batch_size=64, validation_split=0.2)
test_loss, test_acc = model2.evaluate(big_test, big_test_labels)

Epoch 1/4
313/313 [==============================] - 94s 300ms/step - loss: 0.0372 - accuracy: 0.9882 - val_loss: 0.5207 - val_accuracy: 0.8670
Epoch 2/4
313/313 [==============================] - 86s 275ms/step - loss: 0.0186 - accuracy: 0.9941 - val_loss: 0.5463 - val_accuracy: 0.8630
Epoch 3/4
313/313 [==============================] - 86s 276ms/step - loss: 0.0141 - accuracy: 0.9951 - val_loss: 0.6721 - val_accuracy: 0.8578
Epoch 4/4
782/782 [==============================] - 21s 27ms/step - loss: 0.7805 - accuracy: 0.8421


I mean yeah it's a bit overfitted but let's just run with it.

In [481]:
original_string = gpt_negative_reviews[1]
new = ' '.join(word for word in original_string.split() if word not in stop_words)
predict2([new])

1/1 [==============================] - 0s 39ms/step
[[0.36603954]]
[0, 18837, 5474, 0, 1030, 4659, 676, 43, 27, 18431, 0, 3715, 409, 53, 57, 88, 6108, 0, 0, 0, 1589, 2082, 27, 39581, 0, 228, 2402, 6027, 32, 16565, 0, 0, 4404, 13733, 40416, 197, 8231, 230, 0, 18837, 5474, 14078, 32, 12108, 580, 69175, 57, 88, 11958, 55403, 0, 0, 4809, 2364, 98, 5474, 0, 5945, 0, 0, 33859, 12930, 227, 36, 0, 4049, 15480, 19317, 743, 343, 0, 0, 3554, 734, 0, 18837, 5474, 2129, 6318, 4006, 0, 58605, 172, 10930, 4068, 2555, 0, 0, 1700, 486, 314, 73, 27, 0, 6453, 1338, 2119, 2838, 3274, 41, 0, 0, 71, 44813, 0, 18837, 5474, 1578, 1810, 3247, 41, 91, 20201, 0, 72077, 98, 586, 1141, 10214, 0, 0, 4404, 278, 13599, 5342, 0, 1279, 27, 0, 0, 18837, 0, 227, 36, 34553, 278, 0, 1051, 53, 4049, 4182, 1071, 197, 0, 0, 7523, 717, 1612, 0, 42, 4736, 37488, 365, 2823, 0, 4914, 0, 18837, 5474, 1193, 1642, 91, 4928, 314, 69, 7530, 2151, 4435, 0, 0, 18431, 1096, 256, 2984, 24282, 734, 39591, 0]
The tandem bike I recently purc

We did successfully labeled the tandem bike! What about the tomato soup?

In [482]:
original_string = gpt_negative_reviews[0]
new = ' '.join(word for word in original_string.split() if word not in stop_words)
predict2([new])

1/1 [==============================] - 0s 55ms/step
[[0.9068725]]
[0, 67, 10370, 5769, 1329, 582, 36, 52, 0, 0, 2336, 1914, 33874, 1691, 943, 7859, 0, 6964, 32, 13181, 1901, 45235, 1193, 7976, 15356, 28, 6431, 36, 10370, 0, 0, 0, 5129, 6894, 7483, 874, 2146, 1731, 5047, 1520, 5614, 38355, 32, 3816, 13113, 10370, 0, 0, 10027, 1034, 1280, 0, 3671, 4627, 0, 1134, 28, 19822, 486, 10370, 0, 0, 12989, 0, 5769, 418, 18290, 4167, 32795, 39199, 141, 10179, 138, 353, 0, 0, 15550, 3887, 31155, 25155, 676, 43, 27, 2688, 0, 13517, 114, 6799, 441, 0, 0, 11719, 3485, 210, 10474, 0, 840, 2217, 314, 73, 27, 0, 0, 18078, 8846, 0, 1197, 32, 4006, 8424, 0, 0, 7523, 717, 1612, 0, 42, 4736, 2830, 365, 2823, 0, 4914, 0, 67, 10370, 5769, 1193, 906, 57, 88, 1118, 0, 0, 18431, 1096, 731, 227, 343, 33874, 582, 28, 59, 437, 0]
Opening can tomato soup disappointing experience from very start. The promise delightful culinary treat quickly faded I encountered an overwhelmingly bland aroma failed evoke richness one e

uhhhh

| Epochs | Review item | Prediction | Confidence
-------- | ----------- | -------- | --------
4 Epochs | Negative Tomato Soup Review | Negative | 27.78%
8 Epochs | Negative Tomato Soup Review | Positive | 90.69%
4 Epochs | Negative Tandem Bike Review | Positive | 80.41%
8 Epochs | Negative Tandem Bike Review | Negative | 36.60%

Hmm. I'll simply reason like this: Reviews for movies and tandem bikes are simply just not similar enough. And uh, to prove this. We will predict on actual IMDB reviews from 2023 (to be sure they aren't in the training data).

In [532]:
review_1_star = [
'''My review is not based on the entire movie because I only managed to get through about 1 hour and 20 minutes before I finally lost the will to live. There is no way I could have put myself through another nearly 2 hours. I can't believe this movie gets so many high ratings? Why? It really is the most boring drivel I've ever watched (well for what I managed to watch). I knew what it was about, I knew it had a very good cast but I was expecting a lot more from it. However, its just so slow and boring and I just completely lost interest. I have no doubt it will probably win loads of awards/oscars etc. But all those awards are a farce as has been proven with various winners in recent years. Luckily I managed to watch this movie from the comfort of my home and I can only feel sorry for the people who went to the cinema to watch this. I can imagine there a lot of people who must have fell asleep half way through it. I'm just being honest and trying to save people from wasting their time. Don't trust those other reviews making out that this movie is a masterpiece because its one of the most boring movies I've ever seen.''',
'''Went to see this in IMAX last night with my son. He's been wanting to see it since it came out he loves history, I love physics, Engineering, etc. So booked... well with high expectations came huge let downs. The first hour or eternity as it felt was jarring jumping from one time line to another, the acting was adequate but the script was well all over the place. Then came the 2nd hour, now this was the only bit I liked the making of said Bomb. But again the throwing in of just ear breaking noise just to fill in time. Then the last hour we'll this is where I actually fell asleep. It was not needed it was long and boring with very poor acting from all. I really really can't see why people are raving about this. When I saw Nolan was directing I thought great.. wow I was wrong. I left feeling drained but from being utterly bored and let down. I'm still waiting for a decent film to come out something that we haven't had for a number of years now, this was not it. Just to add we know that hard hitting historical can be done and done well, Schindler's list being one! I have read through a lot of the 1 star reviews and they seem to all be written by fans of Nolan's, like them i enjoyed interstellar (maybe not the last 20 mins where it went off on a tangent).''',
'''This movie got the ADHD!!!! Nothing happens but the scenes keep on changing every 2-3 seconds with some very epic battle music in the background! Ok ok, I get it, it's a historical drama so some tweaks has to be made so we don't fall asleep in the 1st 10 minutes of the movie, even so, where's the character building and the drama? The delivery is weak and the movie is epileptic seizure inducing worthy of the TIKTOK generation era. If you just change the angle of the same scene every 2 seconds that doesn't mean you're making it more interesting, you're just making junk for the attention span of a teenage tiktok user. The rating on this movie is delusional, this movie is a 2/5 stars tops, for the effort and the actors who did bring the shine to this garbage.''',
'''I had one more chance for Nolan after the garbage Tenet. And i have none now. This movie shows you how money talks in Hollywood. There is nothing good about this movie, there isn't any theme or clear conflict of the plot even. Everything is mixed together in this bore- fest auto biographic film. Its scenes are ripped off from other films and this film pretends it is bigger than it is with its over the top sound design. I don't recommend anyone to waste their time and money to this. No characther development, nothing you can relate. Soulless, passionless filmmaking. I couldn't believe what i was seeing.''',
'''After you watch Oppenheimer and leave the movie theater, you ask yourself: "What was the point of it?" Unfortunately the answer is not clear at all. I strongly beleive that a movie about the scientist who played a key role in the creation of the atomic bomb should unquestionably emphasize on the consequences of this invention and send strong peaceful messages to the audience! Especially in the world we live in today! However, it looks like in the new movie Oppenheimer they almost ignore the tragic facts that hundreds of thousands of innocent people died from the first 2 bombs and that the whole world changed after Oppenheim's invention, which was obviously the biggest drama in this one human being's life. They emphasize and build the story in the movie around the fact that Oppenheimer was later accused of being a spy of the Soviets (who were allies to the Americans in WW2...). I feel sad that this will be an award winning movie and that most people in the audience will say (or feel they have to say) "A! Oh!! Great movie!", because the director, writers, cast are famous and the topic is important - all these superlatives about a somewhat boring movie, decorated with some nudity (not clear either why this was needed and related to the topic) and overall pointless movie... Or, at least, much more pointless than it should be!'''
]

review_10_star = [
"""[START] I'm still collecting my thoughts after experiencing this film, Cillian Murphy might as well start clearing a space on his mantle for the Best Actor Oscar.

This film is a masterclass in weaving narratives and different time periods while exploring the profound depths of a man whose actions altered the world's trajectory forever, for better or worse. Nolan brings us into the complexities of Oppenheimer, and all the moral conflicts stirring within him.

Murphy's portrayal is so riveting that the long run-time became an afterthought. Robert Downey Jr also offers a great performance and Nolan's push and pull with how he uses sound design throughout is the cherry on top.

Some viewers might need a brief refresher on WWII and Cold War history, but any film lover should be happy to willingly lose themselves in this film for hours on end.""",

"""Oppenheimer might be the best film I watched in a long, long time.

Very different than Nolan's recent films, especially the Sci-Fi ones, but shows that Nolan can master the Biopic/Drama genre just as well as he can any other genre he tried to tackle yet.

The film is 3-hours long but goes through very quickly and enjoyably. Without spoiling anything, the film presents important and very relevant subjects, and doing so while being non-stop entertainment and a comprehensive character study and a study of our society on a very high pace.

Without mentioning anything specific, there was one scene that caused almost every single person in the theatre to move nervously in the seats, non-stop for a long period of time, being one of the most intense scenes I ever watched in a movie and reminding me of the true power of the cinematic experience like no other movie did in recent years.

The year is only half-way through but right now this is my top pick for the upcoming awards season. Picture, Writing, Directing, Acting, Score-- Oppenheimer is a winner on all fronts. A rare feat for filmmaking and a salient reminder that cinema is not dead.

I highly recommend this film to everyone. Watched it once already, and going back to the theatre for at least a few more times soon.""",

"""I may consider myself lucky to be alive to watch Christopher Nolan Works which get better by years.

Oppenheimer is - with no doubt- going to be one of the best movies in the history. Amazing cinematography, Exceptional acting and terrifying Soundtracks.

All the cast are great from cilian Murphy who is going for the oscar with this role to Rupert Downey jr and Emily blunt and finally rami malik who has small scenes but you will never forget them.

I didn't watch it in Imax as i couldn't wait and ran to the nearest cinema but now i will sure book an imax ticket.

Don't waste any time, book your ticket and Go watch it.. NOW.""",

"""This movie is just... wow! I don't think I have ever felt like this watching a movie! Its like a blend of being sad but also scared! I read that Christopher Nolan said it kind of had themes of horror, and watching the movie i think I knew what he meant! Very few movies can make you feel quite like this one can!

Nolan once again shows he is an expertly craftsman in filmmaking! This stands as perhaps one of his more humble movies but also one of his greatest! Reminds me of his earlier movies!

The cast is also AMAZING with Cillian Murphy delivering the performance of his carrer as Oppenheimer, esentially becoming him, and pretty much securing himself an Oscar nomination for best lead actor! Robert Downey Junior also gives one of his best performances, reminding us all that despite 10 years as Iron man, he can still act!

The soundtrack, sound and editing is also masterfull and further creates a cinematic experience like no other!

Overall an esential viewing experience about historic events that still remains very relevant to this day! One of my favorite Nolan movies!""",

"""This movie is very interesting and very thrilling. Since this movie had no action and was mostly just a documentary and was 3 hours long, I though that it was going to be boring. But, the 3 hours went by very fast and had me at the edge of my seat the whole time. This movie is like no other movie I had ever seen it is very unique and mind blowing. The cinematography is beautiful and the aesthetic of the movie is also beautiful. Anyone who is interested in the history of war and bombs would love this movie but I think anyone would enjoy this movie. Oppenheimer is one of the best movies I have seen this decade."""]

review_5_star  = [
"""This must be the most overrated film of the year.

Like every other typical American biographical movie, it glorifies its subject. According to Nolan, Oppenheimer is the most important person who ever lived. Really?

The movie contains at least 1.5 hours of uninteresting courtroom drama.

There are far too many characters.

Not a single compelling dialogue about the moral impact of the bomb.

The continuous music propels the film, but as a result there is zero emotional impact when needed.

Excellent acting by all, though, and occasional nice directorial effects.

Most ridiculous moment of the film: While having sex with Oppenheimer, Florence Pughs character randomly selects a sentence in a book in Sanskrit which just happens to be the infamous "Now I Am Become Death, the Destroyer of Worlds" quote.""",

"""Yes, the acting for the most part is excellent. However, the movie/story does a grave injustice to the portrayal of a brilliant and complex human being. It presents a poorly edited one dimensional view of his life, a disjointed story dwelling mostly on political issues and travails. There was very little portrayal of his leadership skills in assembling brilliant minds to solve extremely complex, complicated issues. Overall, a missed opportunity to present the true richness of this incredible mind to the world. The editing seemed calculated to provide some of the actors with more screen time, and an opportunity to demonstrate their acting. By the end, I found myself wondering when it would end.""",

"""Overambitieus movie with too many stories to tell that does not manage to tell any of the stories really well because of it. I truly don't understand the high ratings. I was bored to death the last hour with the whole overextended communist drama. Also, the was little character building. Instead the viewer is thrown from scene to scene for the first ninety minutes. The movie doesn't allow you to draw you along further in any scene than surface level which makes many scenes shallow. I felt like sitting through church when I was a little boy. I wanted to leave but I couldn't. I feel mentally raped.""",

"""An interesting man to create a movie about and Cillian Murphy did great job portraying him, but obviously the main appeal of the story is how he created the atomic bomb. Did they show what went into that project? Sure, but it wasn't the main focus. The majority of the movie was much more about Oppenheimer's personal life, including his political views and relationship issues. I also didn't care for the way it kept jumping around in time, sort of like the recent Elvis movie. This creates a sense that you're watching a trailer for a movie, instead of a movie itself. If it was half as long and was more about the bomb, I would have rated it much higher.""",

"""I studied Quantum Physics and I knew Oppenheimer biography before watching this movie so expectations were high. Unfortunately, full 3 hours do not make this much better than trailer - I learned nothing new. Outstanding acting from Cillian Murphy and RDJ, Emily was just blunt, so was Matt Damon's performance. Soundtrack is mediocre although delayed explosion sound made few people jump from their seats. Bomb making is virtually non-existent and there is a a lot of chat in the film, typical for a documentary. If you are late 1 hour for cinema screening, you won't miss anything it is that bad. I probably will never watch this again."""

]

In [537]:
def predict3(reviews):
    for i in range(len(reviews)):
        new_text = reviews[i].lower()
        new_text = ' '.join(word for word in new_text.split() if word not in stop_words)

        new_sequence = [imdb.get_word_index().get(word, 0) for word in new_text.split()]
        print(new_sequence)
        print(new_text)


        
        # # new_sequence = [index if index < 20040 else 0 for index in new_sequence]

        padded = pad_sequences([new_sequence], maxlen=200)
        print(padded)
        # print(padded)
        prediction = model2.predict(padded)
        # # print(prediction)

        sentiment = "Positive" if prediction[0, 0] > 0.5 else "Negative"
        print(new_sequence)
        # print(reviews[i])
        print(f"Predicted sentiment: {sentiment} (Confidence: {prediction[0, 0] * 100:.2f}%)")
        print()

In [538]:
predict3(review_1_star)

[58, 730, 21, 445, 433, 85, 61, 1316, 76, 140, 41, 297, 531, 888, 231, 156, 414, 413, 77, 0, 47, 54, 93, 97, 25, 273, 543, 140, 157, 751, 238, 0, 188, 261, 211, 35, 108, 309, 0, 0, 63, 88, 354, 3569, 204, 123, 293, 0, 48, 1316, 0, 694, 48, 0, 694, 66, 52, 49, 174, 1014, 173, 50, 36, 0, 0, 91, 40, 35, 547, 354, 40, 337, 413, 0, 25, 54, 821, 77, 239, 1173, 4300, 0, 0, 29, 145, 2125, 23, 3683, 44, 74, 16575, 995, 8724, 1133, 0, 3463, 1316, 103, 36, 5093, 58, 341, 67, 61, 232, 803, 81, 34, 432, 435, 103, 0, 67, 835, 47, 173, 81, 34, 212, 25, 1580, 2356, 317, 93, 140, 0, 143, 40, 109, 1199, 266, 604, 81, 36, 3118, 65, 0, 89, 1681, 145, 82, 854, 228, 43, 988, 85, 91, 28, 88, 354, 99, 204, 123, 0]
my review not based entire because only managed get through about 1 hour 20 minutes before finally lost will live. there no way could have put myself through another nearly 2 hours. can't believe gets so many high ratings? why? really most boring drivel i've ever watched (well what managed watch). k

Failed all.

In [ ]:
predict3(review_5_star)

1/1 [==============================] - 0s 40ms/step
Predicted sentiment: Positive (Confidence: 96.22%)

1/1 [==============================] - 0s 51ms/step
Predicted sentiment: Positive (Confidence: 80.77%)

1/1 [==============================] - 0s 40ms/step
Predicted sentiment: Negative (Confidence: 0.09%)

1/1 [==============================] - 0s 42ms/step
Predicted sentiment: Positive (Confidence: 80.51%)

1/1 [==============================] - 0s 42ms/step
Predicted sentiment: Negative (Confidence: 19.02%)



Correctly classified like 0.5/5 as average

In [ ]:
predict3(review_10_star)

1/1 [==============================] - 0s 42ms/step
Predicted sentiment: Positive (Confidence: 100.00%)

1/1 [==============================] - 0s 40ms/step
Predicted sentiment: Negative (Confidence: 2.73%)

1/1 [==============================] - 0s 41ms/step
Predicted sentiment: Negative (Confidence: 15.16%)

1/1 [==============================] - 0s 40ms/step
Predicted sentiment: Positive (Confidence: 81.82%)

1/1 [==============================] - 0s 42ms/step
Predicted sentiment: Positive (Confidence: 89.98%)



Correctly classified 3/5 as positive

Maybe the last 4 epochs overfitted. I will have to find out.